In [1]:
library(DESeq2)
library(EnhancedVolcano)
library(clusterProfiler)
library(enrichplot)
library(ggplot2)
organism = "org.Mm.eg.db"
library(organism, character.only = TRUE)
library(DOSE)

Warning message:
“package ‘DESeq2’ was built under R version 4.3.3”
Loading required package: S4Vectors

Warning message:
“package ‘S4Vectors’ was built under R version 4.3.3”
Loading required package: stats4

Loading required package: BiocGenerics

Warning message:
“package ‘BiocGenerics’ was built under R version 4.3.2”

Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min



Attaching package: ‘S4Vectors’


The following object is masked from ‘package:utils’:

    find

In [2]:
outdir = "../chapters/4_results_and_discussion/figures/dea/deseq2/"

In [3]:
df <- read.table("dea/sum.tsv", header=TRUE, sep="\t")
# Drop duplicates in id column
df <- df[!duplicated(df$id),]
rownames(df) <- df$id
df$id <- NULL

annotation <- read.table("dea/annotation.bed", header=FALSE, sep="\t", col.names = c("chr", "start", "end", "name", "score", "strand", "type", "gene.id", "gene", "transcripts", "databases"))
# Remove chr, start, end, score, strand, gene.id, transcripts
annotation$chr <- NULL
annotation$start <- NULL
annotation$end <- NULL
annotation$score <- NULL
annotation$strand <- NULL
annotation$gene.id <- NULL
annotation$transcripts <- NULL
annotation$has_db <- annotation$databases != "."

rownames(annotation) <- annotation$name
annotation$name <- NULL

phenotype <- read.csv("dea/phenotype.csv", header=TRUE, row.names = 1)
phenotype$transgene <- as.factor(phenotype$transgene)
phenotype$drug <- as.factor(phenotype$drug)

# Center and scale age and induction
phenotype$age <- scale(phenotype$age)
phenotype$induction <- scale(phenotype$induction)

df_genes <- read.table("dea/gene_tpm.tsv", sep="\t", header=TRUE, row.names=1)
df_genes$gene_name <- NULL
# Order genes columns like phenotype rows
df_genes <- df_genes[,rownames(phenotype)]

phenotype$esr1 <- scale(as.numeric(df_genes["Esr1",]))

In [4]:
run_analysis <- function(title, phenotype = NULL, design = NULL, dds = NULL, contrast = NULL, name = NULL) {
    directory <- paste0(outdir, sub(" ", "_", tolower(title)))
    # Fail if both contrast and name are NULL
    if (is.null(contrast) && is.null(name)) {
        stop("Both contrast and name cannot be NULL")
    }
    # Fail if none of contrast and name are NULL
    if (!is.null(contrast) && !is.null(name)) {
        stop("Both contrast and name cannot be provided")
    }

    if (is.null(phenotype) && is.null(design) && is.null(dds)) {
        stop("Either phenotype and design or dds must be provided")
    }

    if (is.null(phenotype) != is.null(design)) {
        stop("Both phenotype and design must be provided")
    }

    if (!is.null(dds) && !is.null(phenotype)) {
        stop("Either dds or phenotype and design must be provided")
    }

    alpha <- 0.05

    if (is.null(dds)) {
        dds <- DESeqDataSetFromMatrix(countData = df[, rownames(phenotype)],
                                    colData = phenotype,
                                    design = design)
        dds <- DESeq(dds)
    }

    if (!is.null(contrast)) {
        res <- results(dds, contrast = contrast, alpha = alpha, lfcThreshold = 2, altHypothesis = "greaterAbs")
    } else {
        res <- results(dds, name = name, alpha = alpha, lfcThreshold = 2, altHypothesis = "greaterAbs")
    }
    res <- res[order(res$padj),]
    res <- cbind(res, annotation[rownames(res),])

    # Recreate directory if it exists
    if (dir.exists(directory)) {
        unlink(directory, recursive = TRUE)
    }
    dir.create(directory, showWarnings = FALSE, recursive = TRUE)

    write.table(res, file=paste0(directory, "/res.tsv"), sep="\t", col.names=NA, row.names=TRUE, quote=FALSE)

    colors <- ifelse(res$has_db, "red", "black")
    names(colors) <- ifelse(res$has_db, "Has database", "No database")

    EnhancedVolcano(res,
        x='log2FoldChange',
        y='padj',
        lab=res$gene,
        title="DESeq2",
        colCustom=colors,
        drawConnectors=FALSE,
        pCutoff = alpha)
    ggsave(paste0(directory, "/volcano.png"))

    # Keep only rows without any NA
    res <- res[complete.cases(res),]

    sign <- res[res$padj < alpha,]

    genes <- unique(unlist(strsplit(sign$gene, ",")))
    write.table(genes, file=paste0(directory, "/genes.txt"), row.names=FALSE, col.names=FALSE, quote=FALSE)
    db <- unique(unlist(strsplit(sign$databases, ",")))
    write.table(db, file=paste0(directory, "/db.txt"), row.names=FALSE, col.names=FALSE, quote=FALSE)

    gene_list <- sign$log2FoldChange
    names(gene_list) <- sign$gene
    gene_list = sort(gene_list, decreasing = TRUE)

    gse <- gseGO(geneList=gene_list,
            ont ="ALL", 
            keyType = "SYMBOL",
            pvalueCutoff = 0.05,
            verbose = TRUE,
            OrgDb = organism,
            pAdjustMethod = "BH")

    # If no significant GO terms, return
    if (nrow(as.data.frame(gse)) > 0) {
        dotplot(gse, showCategory=10, split=".sign") + facet_grid(.~.sign)
        ggsave(paste0(directory, "/dot.png"))

        gse <- pairwise_termsim(gse)
        emapplot(gse, showCategory = 10)
        ggsave(paste0(directory, "/emap.png"))

        cnetplot(gse, categorySize="pvalue", foldChange=gene_list, showCategory = 3)
        ggsave(paste0(directory, "/cnet.png"))

        ridgeplot(gse) + labs(x = "enrichment distribution")
        ggsave(paste0(directory, "/ridge.png"))
    } else {
        print("No significant GO terms")
    }

    return(dds)
}

# Aging

In [5]:
dds_esr1 <- run_analysis(
    "Age",
    phenotype = phenotype,
    design = ~ age + transgene + induction + drug + esr1,
    name = "age"
)

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

15 rows did not converge in beta, labelled in mcols(object)$betaConv. Use larger maxit argument with nbinomWaldTest

Saving 6.67 x 6.67 in image
preparing geneSet collections...

GSEA analysis...

Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (83.56% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are duplicate gene names, fgsea may produce unexpected results.”
leading edge analysis...

done...

Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image
Warning message in cnetplot.enrichResult(x, ...):
“Use 'color.params = list(foldChange = your_value)' instead of 'foldChange'.
 The foldChange 

# ESR1 association

In [6]:
run_analysis(
    "ESR1",
    dds = dds_esr1,
    name = "esr1"
)

Saving 6.67 x 6.67 in image
preparing geneSet collections...

GSEA analysis...

Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (87.45% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are duplicate gene names, fgsea may produce unexpected results.”
leading edge analysis...

done...

Saving 6.67 x 6.67 in image
Saving 6.67 x 6.67 in image
Warning message in cnetplot.enrichResult(x, ...):
“Use 'color.params = list(foldChange = your_value)' instead of 'foldChange'.
 The foldChange parameter will be removed in the next version.”
Scale for size is already present.
Adding another scale for size, which will replace the existing scale.
Saving 6.67 x 6.67 in image
Warning message:
“ggrepel: 770 unlabeled data points (too many overlaps). Consider increasing max.over

class: DESeqDataSet 
dim: 56620 72 
metadata(1): version
assays(4): counts mu H cooks
rownames(56620): chr1:3729265-3729444:+ chr1:4212834-4337843:- ...
  chrX_GL456233v2_random:409325-409604:-
  chrX_GL456233v2_random:409325-422124:-
rowData names(42): baseMean baseVar ... deviance maxCooks
colnames(72): aging_12m_ESR1_no_1 aging_12m_ESR1_no_2 ...
  antiHormonal_18m_ESR1_no_2 antiHormonal_18m_ESR1_no_3
colData names(8): condition age ... esr1 sizeFactor

# Drug effects

## Tamoxifen

In [7]:
dds_no_esr1 <- run_analysis(
    "Tamoxifen",
    phenotype = phenotype,
    design = ~ age + transgene + induction + drug,
    contrast=c("drug", "tamoxifen", "no")
)

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

Saving 6.67 x 6.67 in image
preparing geneSet collections...

GSEA analysis...

Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (52.48% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are duplicate gene names, fgsea may produce unexpected results.”
no term enriched under specific pvalueCutoff...



[1] "No significant GO terms"


## Letrozole

In [8]:
run_analysis(
    "Letrozole",
    dds = dds_no_esr1,
    contrast=c("drug", "letrozole", "no")
)

Saving 6.67 x 6.67 in image
preparing geneSet collections...

GSEA analysis...

Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (64.39% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are duplicate gene names, fgsea may produce unexpected results.”
no term enriched under specific pvalueCutoff...



[1] "No significant GO terms"


class: DESeqDataSet 
dim: 56620 72 
metadata(1): version
assays(4): counts mu H cooks
rownames(56620): chr1:3729265-3729444:+ chr1:4212834-4337843:- ...
  chrX_GL456233v2_random:409325-409604:-
  chrX_GL456233v2_random:409325-422124:-
rowData names(38): baseMean baseVar ... deviance maxCooks
colnames(72): aging_12m_ESR1_no_1 aging_12m_ESR1_no_2 ...
  antiHormonal_18m_ESR1_no_2 antiHormonal_18m_ESR1_no_3
colData names(8): condition age ... esr1 sizeFactor